# RAG - Lab Teknik Informatika ITS
**Retrieval-Augmented Generation** menggunakan data laboratorium Departemen Teknik Informatika ITS

**Stack:**
- Embedding: `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` (mendukung Bahasa Indonesia)
- Vector Store: `FAISS`
- LLM: `llama-3.3-70b-versatile` via **Groq API** (128k context, sangat cepat)
- Framework: `LangChain`

## 📦 1. Install Dependencies

In [1]:
!pip install -q langchain langchain-community langchain-huggingface langchain-groq
!pip install -q langchain-google-genai
!pip install -q faiss-cpu sentence-transformers huggingface_hub
print('✅ Semua library berhasil diinstall!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.

## 2. Setup Groq API Key

In [2]:
# import os
# from google.colab import userdata

# # Baca GROQ API Key dari Colab Secrets
# # Pastikan kamu sudah menambahkan secret bernama 'GROQ_API_KEY' di panel kiri (ikon kunci)
# GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# print(f'Groq API Key berhasil dimuat! ({GROQ_API_KEY[:8]}...)')


Groq API Key berhasil dimuat! (gsk_CcaY...)


In [37]:
# ── Setup Gemini API Key ────────────────────────────────────────
import os
from google.colab import userdata

# Pastikan kamu sudah menambahkan secret bernama 'GEMINI_API_KEY'
# di panel kiri Colab (ikon kunci 🔑).
# Atau ganti langsung: GEMINI_API_KEY = 'your-api-key-here'
GEMINI_API_KEY = userdata.get('GEMINI_API')

os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

print(f'Gemini API Key berhasil dimuat! ({GEMINI_API_KEY[:8]}...)')

Gemini API Key berhasil dimuat! (AIzaSyDB...)


## 📂 3. Upload dan Load Data JSON
Upload file `data_lab_informatika_its.json` ke Colab, atau gunakan data embedded di bawah.

In [38]:
# Opsi A: Upload file dari lokal
from google.colab import files
print('Upload file data_lab_informatika_its.json:')
uploaded = files.upload()

Upload file data_lab_informatika_its.json:


In [39]:
import json

with open('data_lab_informatika_its.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"✅ Data berhasil dimuat!")
print(f"📊 Jumlah Laboratorium: {len(data['laboratorium'])}\n")
for lab in data['laboratorium']:
    aliases = ', '.join(lab.get('alias', [])[:3])
    print(f"  • [{lab['singkatan']:7}] {lab['nama']}")
    print(f"           Alias: {aliases}")

✅ Data berhasil dimuat!
📊 Jumlah Laboratorium: 8

  • [RPL    ] Laboratorium Rekayasa Perangkat Lunak
           Alias: RPL, Lab RPL, Software Engineering
  • [KBJ    ] Laboratorium Komputasi Berbasis Jaringan
           Alias: KBJ, NCC, Net Centric Computing
  • [KCV    ] Laboratorium Komputasi Cerdas dan Visi
           Alias: KCV, Lab KCV, Intelligent Computing and Vision
  • [NETICS ] Laboratorium Teknologi Jaringan dan Keamanan Siber Cerdas
           Alias: NETICS, Lab NETICS, AJK
  • [GIGa   ] Laboratorium Grafika, Interaksi, Gim dan Analitik
           Alias: GIGa, GIGA, Lab GIGa
  • [AP     ] Laboratorium Algoritma dan Pemrograman
           Alias: AP, Lab AP, Algorithm and Programming
  • [MCI    ] Laboratorium Manajemen Cerdas Informasi
           Alias: MCI, Lab MCI, Intelligent Information Management
  • [PKT    ] Laboratorium Pemodelan dan Komputasi Terapan
           Alias: PKT, Lab PKT, Applied Modelling and Computation


## 📄 4. Konversi Data JSON ke Dokumen (Chunking)
Setiap laboratorium dikonversi menjadi beberapa chunk dokumen yang kaya konteks, cocok untuk RAG.

In [27]:
from langchain_core.documents import Document

def lab_to_documents(data: dict) -> list:
    """Konversi setiap lab menjadi 5 chunk semantik."""
    documents = []
    institusi  = data['institusi']
    departemen = data['departemen']

    for lab in data['laboratorium']:
        nama      = lab['nama']
        singkatan = lab['singkatan']
        kepala    = lab['kepala_laboratorium']
        lokasi    = lab['lokasi']
        alias_str = ', '.join(lab.get('alias', [singkatan]))
        nama_lama = lab.get('nama_lama', '-')
        meta      = {"lab": singkatan, "nama_lab": nama}

        # ── Chunk 1: Profil ──────────────────────────────────
        documents.append(Document(
            page_content=f"""Laboratorium    : {nama} ({singkatan})
Nama Inggris    : {lab.get('nama_inggris', '-')}
Alias/Nama Lain : {alias_str}
Nama Lama       : {nama_lama}
Departemen      : {departemen} — {institusi}
Lokasi          : {lokasi}
Website         : {lab.get('website', '-')}
Kepala Lab      : {kepala['nama']} | Email: {kepala['email']} | Inisial: {kepala['inisial']}
Deskripsi       : {lab.get('deskripsi', '-')}""",
            metadata={**meta, "tipe": "profil"}))

        # ── Chunk 2: Dosen ───────────────────────────────────
        dosen_lines = [
            f"  • {d['nama']} ({d.get('jabatan','Anggota')}) — {d['email']}"
            + (f" | Riset: {d['bidang_riset']}" if d.get('bidang_riset') else "")
            for d in lab['dosen_anggota']
        ]
        documents.append(Document(
            page_content=f"""Dosen Anggota Lab {nama} ({singkatan}) — {alias_str}:
{chr(10).join(dosen_lines)}
Total: {len(lab['dosen_anggota'])} dosen.""",
            metadata={**meta, "tipe": "dosen"}))

        # ── Chunk 3: Mata Kuliah & Bidang Keahlian ───────────
        documents.append(Document(
            page_content=f"""Mata Kuliah & Bidang Keahlian Lab {nama} ({singkatan}) — {alias_str}:
Bidang Keahlian:
{chr(10).join('  • ' + b for b in lab.get('bidang_keahlian', []))}
Mata Kuliah:
{chr(10).join('  • ' + mk for mk in lab.get('mata_kuliah', []))}""",
            metadata={**meta, "tipe": "mata_kuliah"}))

        # ── Chunk 4: Fasilitas ───────────────────────────────
        fasilitas = lab.get('fasilitas', {})
        catatan   = fasilitas.get('catatan_khusus', '')
        documents.append(Document(
            page_content=f"""Fasilitas Lab {nama} ({singkatan}) — {departemen}:
{chr(10).join('  • ' + item for item in fasilitas.get('komputer', []))}
{'📌 Catatan: ' + catatan if catatan else ''}""",
            metadata={**meta, "tipe": "fasilitas"}))

        # ── Chunk 5: Asisten 2022-2023 ───────────────────────
        asisten = lab.get('asisten_2022_2023', {})
        if asisten:
            ak22 = asisten.get('angkatan_2022', [])
            ak23 = asisten.get('angkatan_2023', [])
            lines_22 = chr(10).join(f"  • {n}" for n in ak22) if ak22 else "  (tidak ada data)"
            lines_23 = chr(10).join(f"  • {n}" for n in ak23) if ak23 else "  (tidak ada data)"
            documents.append(Document(
                page_content=f"""Asisten Laboratorium {nama} ({singkatan}) — Tahun Ajaran 2022/2023:
Angkatan 2022 ({len(ak22)} asisten):
{lines_22}
Angkatan 2023 ({len(ak23)} asisten):
{lines_23}
Total asisten: {len(ak22) + len(ak23)} orang.""",
                metadata={**meta, "tipe": "asisten"}))

    total_labs = len(data['laboratorium'])
    total_docs = len(documents)
    print(f"✅ Total chunks: {total_docs} dokumen")
    print(f"   Lab dengan asisten: {sum(1 for lab in data['laboratorium'] if lab.get('asisten_2022_2023'))} lab")
    return documents

docs = lab_to_documents(data)
print(f"\n── Contoh Chunk Profil ──\n{docs[0].page_content}")
print(f"\n── Contoh Chunk Asisten ──\n{docs[4].page_content}")


✅ Total chunks: 38 dokumen
   Lab dengan asisten: 6 lab

── Contoh Chunk Profil ──
Laboratorium    : Laboratorium Rekayasa Perangkat Lunak (RPL)
Nama Inggris    : Laboratory of Software Engineering
Alias/Nama Lain : RPL, Lab RPL, Software Engineering, Lab Software Engineering, Rekayasa Perangkat Lunak
Nama Lama       : -
Departemen      : Departemen Teknik Informatika — Institut Teknologi Sepuluh Nopember (ITS) Surabaya
Lokasi          : Gedung Departemen Teknik Informatika ITS, Kampus ITS Sukolilo, Surabaya
Website         : https://www.its.ac.id/informatika/laboratorium-rekayasa-perangkat-lunak/
Kepala Lab      : Dr. Sarwosri, S.Kom., MT. | Email: sarwosri@if.its.ac.id | Inisial: SR
Deskripsi       : Laboratorium Rekayasa Perangkat Lunak menawarkan bidang minat yang berfokus pada keahlian melakukan pengujian perangkat lunak, kemampuan mengelola proyek perangkat lunak, kemampuan mengurangi risiko kesalahan perangkat lunak, dan kemampuan membuat perangkat lunak game.

── Contoh Chunk A

## 🧠 5. Buat Embedding & Vector Store (FAISS)

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

print("⏳ Memuat model embedding multilingual...")
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cuda'}  # ganti 'cpu' jika tidak ada GPU
)

print("⏳ Membangun FAISS vector store...")
vectorstore = FAISS.from_documents(docs, embedding_model)
print(f"✅ Vector store siap! {len(docs)} dokumen terindeks.")

⏳ Memuat model embedding multilingual...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


⏳ Membangun FAISS vector store...
✅ Vector store siap! 38 dokumen terindeks.


## 6. Setup LLM via Groq API (llama-3.3-70b-versatile)

In [41]:
# from langchain_groq import ChatGroq

# # Model terbaik Groq untuk RAG:
# #   llama-3.3-70b-versatile  -> terbaik saat ini (konteks 128k, multilingual)
# #   deepseek-r1-distill-llama-70b -> reasoning intensif
# #   llama3-70b-8192          -> stabil, konteks lebih kecil
# llm = ChatGroq(
#     groq_api_key=GROQ_API_KEY,
#     model_name='llama-3.3-70b-versatile',
#     temperature=0.2,
#     max_tokens=1024,
# )

# # Test koneksi
# test = llm.invoke('Halo! Perkenalkan dirimu dalam satu kalimat.')
# print('LLM siap: llama-3.3-70b-versatile (Groq)')
# print(f'Test: {test.content}')


In [45]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

print("Model yang tersedia untuk generateContent:\n")
for model in genai.list_models():
    if "generateContent" in model.supported_generation_methods:
        print(f"  • {model.name}")


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


Model yang tersedia untuk generateContent:

  • models/gemini-2.5-flash
  • models/gemini-2.5-pro
  • models/gemini-2.0-flash
  • models/gemini-2.0-flash-001
  • models/gemini-2.0-flash-exp-image-generation
  • models/gemini-2.0-flash-lite-001
  • models/gemini-2.0-flash-lite
  • models/gemini-2.5-flash-preview-tts
  • models/gemini-2.5-pro-preview-tts
  • models/gemma-3-1b-it
  • models/gemma-3-4b-it
  • models/gemma-3-12b-it
  • models/gemma-3-27b-it
  • models/gemma-3n-e4b-it
  • models/gemma-3n-e2b-it
  • models/gemini-flash-latest
  • models/gemini-flash-lite-latest
  • models/gemini-pro-latest
  • models/gemini-2.5-flash-lite
  • models/gemini-2.5-flash-image
  • models/gemini-2.5-flash-lite-preview-09-2025
  • models/gemini-3-pro-preview
  • models/gemini-3-flash-preview
  • models/gemini-3.1-pro-preview
  • models/gemini-3.1-pro-preview-customtools
  • models/gemini-3-pro-image-preview
  • models/nano-banana-pro-preview
  • models/gemini-3.1-flash-image-preview
  • models/gemin

In [62]:
# ── Setup LLM via Gemini API ────────────────────────────────────────────────
from langchain_google_genai import ChatGoogleGenerativeAI

# Model Gemini yang tersedia:
#   gemini-2.0-flash         -> cepat, hemat quota, cocok untuk RAG
#   gemini-2.0-flash-thinking -> reasoning intensif (lebih lambat)
#   gemini-1.5-pro           -> konteks panjang 2M tokens
#   gemini-1.5-flash         -> cepat & ekonomis
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    google_api_key=GEMINI_API_KEY,
    temperature=0.2,
    max_output_tokens=2048,
)

# Test koneksi
test = llm.invoke('Halo! Perkenalkan dirimu dalam satu kalimat.')
print('LLM siap: gemini-2.0-flash (Google Gemini)')
print(f'Test: {test.content}')

LLM siap: gemini-2.0-flash (Google Gemini)
Test: Halo! Saya adalah model bahasa besar, dilatih oleh Google.


## ⛓️ 7. Bangun RAG Chain

In [63]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""Kamu adalah asisten informasi Laboratorium Teknik Informatika ITS.
Jawab pertanyaan secara langsung, akurat, dan informatif — seperti ChatGPT menjawab pertanyaan faktual.

GAYA MENJAWAB:
- Langsung jawab inti pertanyaannya di kalimat pertama, tanpa salam pembuka.
- Boleh gunakan format poin jika ada beberapa item, tapi tetap berikan penjelasan singkat tiap poin.
- Jika pertanyaan lanjutan, langsung sambung dari konteks sebelumnya tanpa mengulang informasi yang sudah disebutkan.
- Nada: netral-informatif, boleh sedikit kasual tapi tidak berlebihan. JANGAN mulai jawaban dengan "Halo!", "Wah!", atau salam apapun.
- Panjang jawaban: proporsional. Pertanyaan singkat → jawab singkat. Pertanyaan detail → jawab lebih lengkap. Maksimal 150 kata.

PEMETAAN LAB:
- RPL = Rekayasa Perangkat Lunak | KBJ/NCC = Komputasi Berbasis Jaringan
- KCV = Komputasi Cerdas dan Visi | NETICS = Teknologi Jaringan & Keamanan Siber Cerdas
- GIGa/GIGA = Grafika, Interaksi, Gim dan Analitik | AP = Algoritma dan Pemrograman
- MCI = Manajemen Cerdas Informasi | PKT = Pemodelan dan Komputasi Terapan

ATURAN:
- Jawab berdasarkan konteks. Jika ada nama parsial (bu kelly, pak tohari), cari di daftar dosen.
- Jika membandingkan lab, sebutkan bidang keahlian dan contoh mata kuliah masing-masing.
- Jika ada beberapa lab relevan, sebutkan dengan alasan konkret.
- JANGAN berasumsi user adalah dosen atau bagian dari data. User adalah penanya.
- Jika user tanya "siapa saya", jawab: "Saya tidak tahu identitasmu, saya hanya punya data lab dan dosen ITS."
- Jika info tidak ada di konteks, katakan jujur — jangan mengarang.
- Setiap kali merekomendasikan lab, WAJIB sebutkan minimal 1-2 mata kuliah atau
  bidang keahlian SPESIFIK dari lab tersebut yang relevan dengan pertanyaan user.
  Contoh BENAR: "KCV cocok karena ada mata kuliah Data Mining dan Visi Komputer."
  Contoh SALAH: "KCV cocok karena fokus di sistem cerdas." ← terlalu umum
- JANGAN rekomendasikan lab hanya karena dokumennya muncul di konteks.
  Periksa dulu apakah bidang keahlian atau mata kuliah lab itu benar-benar
  menjawab kebutuhan user.


Konteks:
{context}

Pertanyaan: {question}
Jawaban:"""
)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Retriever menggunakan clean question (bukan termasuk history)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 8}
)

def run_rag(inputs):
    """Pisahkan query retrieval (clean) dari query LLM (+ history)."""
    clean_q  = inputs["clean_question"]   # untuk retrieval
    full_q   = inputs["full_question"]    # untuk LLM (sudah + history)

    docs     = retriever.invoke(clean_q)
    context  = format_docs(docs)
    prompt   = PROMPT.format(context=context, question=full_q)
    answer   = llm.invoke(prompt)
    text     = answer.content if hasattr(answer, 'content') else str(answer)

    return {"result": text, "source_documents": docs}

rag_chain = RunnableLambda(run_rag)
print("✅ RAG Chain siap!")

✅ RAG Chain siap!


## 💬 8. Fungsi Query RAG

In [64]:
chat_history = []

def tanya_rag(pertanyaan: str, show_sources: bool = True):
    """Query RAG dengan konteks riwayat percakapan."""
    print(f"\n{'='*60}")
    print(f"❓ Pertanyaan: {pertanyaan}")
    print('='*60)

    # History untuk konteks LLM — retrieval tetap pakai clean question
    if chat_history:
        history_text = "\n".join([
            f"User: {h['user']}\nAsisten: {h['bot']}"
            for h in chat_history[-3:]
        ])
        full_q = f"Riwayat percakapan:\n{history_text}\n\nPertanyaan baru: {pertanyaan}"
    else:
        full_q = pertanyaan

    result = rag_chain.invoke({
        "clean_question": pertanyaan,  # ← retrieval pakai ini (bersih)
        "full_question":  full_q       # ← LLM pakai ini (+ history)
    })

    jawaban = result['result']
    chat_history.append({"user": pertanyaan, "bot": jawaban})

    print(f"\n🤖 Jawaban:\n{jawaban}")

    if show_sources:
        print(f"\n📚 Sumber ({len(result['source_documents'])} dokumen):")
        for i, doc in enumerate(result['source_documents'], 1):
            m = doc.metadata
            print(f"  [{i}] {m.get('nama_lab','?'):45} | tipe: {m.get('tipe','?')}")

    return result

def reset_history():
    global chat_history
    chat_history = []
    print("🔄 Riwayat percakapan direset.")

print("✅ Fungsi tanya_rag() dengan memori siap!")

✅ Fungsi tanya_rag() dengan memori siap!


## 🧪 9. Contoh Query / Uji Coba RAG

In [65]:
reset_history()

tanya_rag("Jika saya suka data dan machine learning, lab mana yang cocok untuk saya?")
tanya_rag("Kenapa tidak MCI?")  # ← sekarang ada konteks dari pertanyaan sebelumnya
tanya_rag("Siapa kepala lab NCC?")
tanya_rag("Apa fasilitas VR yang dimiliki ITS Informatika?")

🔄 Riwayat percakapan direset.

❓ Pertanyaan: Jika saya suka data dan machine learning, lab mana yang cocok untuk saya?

🤖 Jawaban:
Jika Anda menyukai data dan machine learning, Laboratorium Komputasi Cerdas dan Visi (KCV) adalah pilihan yang paling cocok.

*   **Laboratorium Komputasi Cerdas dan Visi (KCV)**: Lab ini berfokus pada "Penerapan metode sistem cerdas" dan "Analisis dan manipulasi data citra". Mata kuliah yang relevan meliputi "Data Mining", "Analisis Data Multivariat", "Visi Komputer", dan "Sistem Temu Kembali Informasi", yang semuanya sangat terkait dengan bidang data dan machine learning.
*   **Laboratorium Pemodelan dan Komputasi Terapan (PKT)**: Lab ini juga bisa menjadi pilihan pendukung karena memiliki bidang keahlian "Peramalan saintifik" dan "Teknik optimasi", serta mata kuliah "Analisis Data Multivariat" yang relevan dengan pengolahan data.

📚 Sumber (8 dokumen):
  [1] Laboratorium Komputasi Cerdas dan Visi        | tipe: mata_kuliah
  [2] Laboratorium Algoritma da

{'result': 'ITS Informatika memiliki beberapa fasilitas *Virtual Reality* (VR) yang tersedia di Laboratorium Grafika, Interaksi, Gim dan Analitik (GIGa), yaitu:\n\n*   **Oculus Rift Set**: Perangkat VR lengkap dengan *game controller*.\n*   **Oculus VR Development KIT DK-2-V51-BDL**: Kit pengembangan VR dari Oculus.\n*   **HTC VIVE Virtual Reality**: Sistem VR dari HTC Vive.\n*   **KAT Walk Plus VR Treadmill**: *Treadmill* khusus untuk pengalaman VR yang imersif.',
 'source_documents': [Document(id='5eb465d4-1d26-4142-860e-f755dd68fb03', metadata={'lab': 'GIGa', 'nama_lab': 'Laboratorium Grafika, Interaksi, Gim dan Analitik', 'tipe': 'fasilitas'}, page_content='Fasilitas Lab Laboratorium Grafika, Interaksi, Gim dan Analitik (GIGa) — Departemen Teknik Informatika:\n  • PC Client: Intel Core i7 Gen 12, RAM 32GB, SSD 1TB, Monitor 24 inci Curved\n  • PC High End: Intel Core i9-12900K, 64GB DDR5, RTX 3080Ti 12GB\n  • Oculus Rift Set (Game controller)\n  • Oculus VR Development KIT DK-2-V51-

## 💬 10. Antarmuka Interactive Chat

In [66]:
# ── Sel 10: Interactive Chat dengan Gradio ─────────────────────────────────
!pip install -q gradio

import gradio as gr

chat_history = []  # reset history

def chat_fn(message, history):
    if not message.strip():
        return "", history

    # History ringkas — hanya untuk konteks LLM, BUKAN untuk retrieval
    if history:
        history_text = "\n".join([
            f"User: {h[0]}\nAsisten: {h[1]}"
            for h in history[-3:]
        ])
        full_q = f"Riwayat singkat:\n{history_text}\n\nPertanyaan sekarang: {message}"
    else:
        full_q = message

    result  = rag_chain.invoke({
        "clean_question": message,   # ← retrieval pakai ini (bersih)
        "full_question":  full_q     # ← LLM pakai ini (+ history)
    })

    history.append((message, result["result"]))
    return "", history




# ── Contoh pertanyaan ──────────────────────────────────────────────────────
examples = [
    "Lab mana yang cocok jika saya suka data dan AI?",
    "Siapa kepala laboratorium NCC / NETICS?",
    "Apa fasilitas VR di Lab GIGa?",
    "Sebutkan semua dosen di lab KCV!",
    "Lab mana yang menawarkan mata kuliah Big Data?",
    "Bagaimana cara reservasi laboratorium NETICS?",
]

# ── Bangun UI Gradio ───────────────────────────────────────────────────────
with gr.Blocks(
    theme=gr.themes.Soft(
        primary_hue="indigo",
        secondary_hue="blue",
        neutral_hue="slate",
        font=gr.themes.GoogleFont("Inter"),
    ),
    title="RAG Lab Informatika ITS",
    css="""
    #header { text-align: center; padding: 20px 0 10px; }
    #header h1 { font-size: 2em; margin-bottom: 4px; }
    #header p  { color: #64748b; font-size: 0.95em; }
    #chatbox   { height: 480px; }
    .badge     { display: inline-block; background: #e0e7ff; color: #4338ca;
                 border-radius: 99px; padding: 2px 10px; font-size: 0.8em;
                 margin: 2px; }
    """
) as demo:

    # ── Header ─────────────────────────────────────────────
    with gr.Column(elem_id="header"):
        gr.Markdown("""
# 🔬 Asisten RAG Lab Informatika ITS
**Retrieval-Augmented Generation** · Data Lab Departemen Teknik Informatika ITS Surabaya
        """)


    # ── Chatbot ────────────────────────────────────────────
    chatbot = gr.Chatbot(
        label="💬 Chat",
        elem_id="chatbox",
        bubble_full_width=False,
        show_copy_button=True,
        avatar_images=(None, "https://www.its.ac.id/informatika/wp-content/uploads/sites/128/2021/02/cropped-logo-its-192x192.png"),
    )

    # ── Input row ──────────────────────────────────────────
    with gr.Row():
        txt = gr.Textbox(
            placeholder="Tanyakan sesuatu tentang lab, dosen, atau fasilitas...",
            label="",
            scale=9,
            container=False,
        )
        btn = gr.Button("Kirim ➤", variant="primary", scale=1, min_width=80)

    # ── Contoh pertanyaan ──────────────────────────────────
    gr.Examples(
        examples=examples,
        inputs=txt,
        label="💡 Contoh pertanyaan:",
    )

    # ── Tombol reset ───────────────────────────────────────
    with gr.Row():
        clear_btn = gr.Button("🗑️ Reset Percakapan", variant="secondary", size="sm")

    # ── Event handlers ─────────────────────────────────────
    btn.click(chat_fn, inputs=[txt, chatbot], outputs=[txt, chatbot])
    txt.submit(chat_fn, inputs=[txt, chatbot], outputs=[txt, chatbot])
    clear_btn.click(lambda: ([], ""), outputs=[chatbot, txt])

    # ── Footer ─────────────────────────────────────────────
    gr.Markdown("""
---
<center><small>🏛️ ITS Surabaya · Teknik Informatika · RAG dengan FAISS + Groq LLaMA 3.1</small></center>
    """)

demo.launch(share=True, quiet=True)

/tmp/ipython-input-524/98384673.py:44: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-524/98384673.py:44: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-524/98384673.py:72: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-524/98384673.py:72: DeprecationWarning: The 'show_copy_button' parameter will be removed in Gradio 6.0. You will need to use 'buttons=["copy"]' instead.
  chatbot = gr.Chatbot(
/tmp/ipython

* Running on public URL: https://0ddba69c7891619bbd.gradio.live


## 💾 11. (Opsional) Simpan & Load Vector Store
Supaya tidak perlu rebuild setiap kali membuka Colab.

In [ ]:
# Simpan vector store ke disk
vectorstore.save_local("faiss_lab_its")
print("✅ Vector store disimpan ke folder 'faiss_lab_its'")

# Download folder vector store
import shutil
shutil.make_archive('faiss_lab_its', 'zip', 'faiss_lab_its')
files.download('faiss_lab_its.zip')

In [ ]:
# Load vector store dari disk (jalankan ini jika mau load ulang)
vectorstore_loaded = FAISS.load_local(
    "faiss_lab_its",
    embedding_model,
    allow_dangerous_deserialization=True
)
print("✅ Vector store berhasil dimuat ulang!")